In [1]:
import pandas as pd

In [2]:
excel_dataset    = "data.xlsx"
pd_excel_dataset = pd.ExcelFile("data.xlsx", engine="openpyxl")


In [3]:
pd_excel_dataset.sheet_names

['SKU MASTER', 'Sales', 'Opening Stock', 'STOCK TRANSFER']

In [4]:
skumaster     = pd.read_excel(pd_excel_dataset, sheet_name = 'SKU MASTER')
sales         = pd.read_excel(pd_excel_dataset, sheet_name = "Sales")
openingstock  = pd.read_excel(pd_excel_dataset, sheet_name = "Opening Stock")
stocktransfer = pd.read_excel(pd_excel_dataset, sheet_name = "STOCK TRANSFER")

## Question 1
For the entire month, what is the total sale value of the game “LTA Wise City”?

In [5]:
sku          = skumaster[skumaster["Description"]=="LTA Wise City"]
sku_code     = sku.iloc[0]["SKU"]
sku_price    = sku.iloc[0]["Price"]
sku_quantity = sales[sales["SKU"] == sku_code ]["Sales"].sum()
print(sku_code, sku_price, sku_quantity, sku_price * sku_quantity)
print("Question 1 Answer: For the entire month, the total sale value of the game “LTA Wise City” is ", sku_price* sku_quantity)

V004 3000 5230 15690000
Question 1 Answer: For the entire month, the total sale value of the game “LTA Wise City” is  15690000


## Question 2
What fraction of total sale quantity (Volume) did “Books” category achieve in the first week? (Jan 1 to Jan 7, both days included) (Float Between 0 to 1)

In [6]:
sales["Date"]     = pd.to_datetime(sales["Date"])
start_date        = pd.Timestamp("2025-01-01")
end_date          = pd.Timestamp("2025-01-07")
first_week_sales  = sales[(sales["Date"] >= start_date) & (sales["Date"] <= end_date)]
book_skus         = skumaster[skumaster["Category"] == "Books"]["SKU"].unique()
books_sales       = first_week_sales[first_week_sales["SKU"].isin(book_skus)]
total_volume      = first_week_sales["Sales"].sum()
books_volume      = books_sales["Sales"].sum()
fraction          = books_volume / total_volume if total_volume != 0 else 0.0
print(f"Question 2 Answer: Books sales volume fraction (Jan 1 to Jan 7): {fraction:.4f}")

Question 2 Answer: Books sales volume fraction (Jan 1 to Jan 7): 0.0977


## Question 3
What is the maximum sale value by a single SKU in a day across all days?
(Sale Value = Sale Qty * Price per Qty)  (INTEGER)


In [7]:
#merged_skumaster_sales                      = pd.merge(skumaster, sales, how="inner", on="SKU")
merged_skumaster_sales                       = pd.merge(sales, skumaster, how="left", on="SKU")
merged_skumaster_sales["Total Sale Value"]   = merged_skumaster_sales["Price"] * merged_skumaster_sales["Sales"]
merged_skumaster_sales.head(5)

,Date,SKU,Product Name,City,Sales,Description,Category,Price,Total Sale Value
0,2025-01-01,F001,Prateek England,Mumbai,23,Prateek England,Fashion,1104,25392
1,2025-01-01,F002,Lucy Phillepe,Mumbai,75,Lucy Phillepe,Fashion,2485,186375
2,2025-01-01,F003,Badlands,Mumbai,33,Badlands,Fashion,1425,47025
3,2025-01-01,F004,Allen Dolly,Mumbai,3,Allen Dolly,Fashion,2640,7920
4,2025-01-01,F005,Bucchi,Mumbai,3,Bucchi,Fashion,11299,33897


In [8]:
missing_skus = merged_skumaster_sales[merged_skumaster_sales["Description"].isna()]
print(missing_skus)
print(missing_skus["SKU"].unique())

Empty DataFrame
Columns: [Date, SKU, Product Name, City, Sales, Description, Category, Price, Total Sale Value]
Index: []
[]


In [9]:
daily_sales                                 = ( merged_skumaster_sales.groupby(["SKU","Description","Date"])["Total Sale Value"].sum().reset_index())
daily_sales.head(5)

,SKU,Description,Date,Total Sale Value
0,B001,Harry putar,2025-01-01,430650
1,B001,Harry putar,2025-01-02,93960
2,B001,Harry putar,2025-01-03,469800
3,B001,Harry putar,2025-01-04,394110
4,B001,Harry putar,2025-01-05,435870


In [10]:
daily_sales_sorted                                = ( merged_skumaster_sales.groupby(["SKU","Description","Date"])["Total Sale Value"].sum().reset_index().sort_values(by="Total Sale Value", ascending=False))
daily_sales_sorted.head(5)

,SKU,Description,Date,Total Sale Value
871,M004,Hotorola,2025-01-04,6215000
873,M004,Hotorola,2025-01-06,5225000
876,M004,Hotorola,2025-01-09,5225000
869,M004,Hotorola,2025-01-02,5005000
778,M001,Jhaomi,2025-01-04,4924800


In [11]:
top_daily_sales                             = daily_sales.sort_values(by="Total Sale Value", ascending=False)
print(top_daily_sales.head(5))

      SKU Description       Date  Total Sale Value
871  M004    Hotorola 2025-01-04           6215000
873  M004    Hotorola 2025-01-06           5225000
876  M004    Hotorola 2025-01-09           5225000
869  M004    Hotorola 2025-01-02           5005000
778  M001      Jhaomi 2025-01-04           4924800


In [12]:
max_row                                     = daily_sales.loc[daily_sales["Total Sale Value"].idxmax()]
print(max_row)

SKU                                M004
Description                    Hotorola
Date                2025-01-04 00:00:00
Total Sale Value                6215000
Name: 871, dtype: object


In [13]:
sku_code                                    = max_row["SKU"]
sku_name                                    = max_row["Description"]
sale_date                                   = max_row["Date"].date()  # just the date part
max_value                                   = int(max_row["Total Sale Value"])

In [14]:
print(f"Highest single-day sale value:")
print(f"SKU: {sku_code}")
print(f"Name: {sku_name}")
print(f"Date: {sale_date}")
print(f"Sale Value: ₹{max_value}")
print(f"Question 3 Answer: Maximum daily sale value by a single SKU: ₹{max_value}")

Highest single-day sale value:
SKU: M004
Name: Hotorola
Date: 2025-01-04
Sale Value: ₹6215000
Question 3 Answer: Maximum daily sale value by a single SKU: ₹6215000


## Question 4
What is the maximum revenue generating category across all days? (STRING)

In [15]:
#merged_skumaster_sales                      = pd.merge(skumaster, sales, how="inner", on="SKU")
merged_skumaster_sales                       = pd.merge(sales, skumaster, how="left", on="SKU")
merged_skumaster_sales["Total Sale Value"]   = merged_skumaster_sales["Price"] * merged_skumaster_sales["Sales"]
merged_skumaster_sales.head(5)

,Date,SKU,Product Name,City,Sales,Description,Category,Price,Total Sale Value
0,2025-01-01,F001,Prateek England,Mumbai,23,Prateek England,Fashion,1104,25392
1,2025-01-01,F002,Lucy Phillepe,Mumbai,75,Lucy Phillepe,Fashion,2485,186375
2,2025-01-01,F003,Badlands,Mumbai,33,Badlands,Fashion,1425,47025
3,2025-01-01,F004,Allen Dolly,Mumbai,3,Allen Dolly,Fashion,2640,7920
4,2025-01-01,F005,Bucchi,Mumbai,3,Bucchi,Fashion,11299,33897


In [16]:
categories_unique = merged_skumaster_sales.Category.unique()
print(categories_unique)
total_revenue = [ merged_skumaster_sales[merged_skumaster_sales["Category"] == cat]["Total Sale Value"].sum() for cat in categories_unique]
print(total_revenue)

['Fashion ' 'Mobiles' 'Books' 'Video Games ' 'Sports / Fitness'
 'Medicines/Healthcare' 'Kitchen ' 'Household  ' 'Pet Supplies' 'Gifting']
[np.int64(77176599), np.int64(282295460), np.int64(20206136), np.int64(38954906), np.int64(66999950), np.int64(32812708), np.int64(51719318), np.int64(23792516), np.int64(62901594), np.int64(83797316)]


In [17]:
cat_revenue = pd.DataFrame({"Category" : categories_unique, "Total Revenue" : total_revenue})
cat_revenue.head(len(cat_revenue))

,Category,Total Revenue
0,Fashion,77176599
1,Mobiles,282295460
2,Books,20206136
3,Video Games,38954906
4,Sports / Fitness,66999950
5,Medicines/Healthcare,32812708
6,Kitchen,51719318
7,Household,23792516
8,Pet Supplies,62901594
9,Gifting,83797316


In [18]:
sorted_cat_revenue = cat_revenue.sort_values(by="Total Revenue", ascending=False)
print(sorted_cat_revenue)

               Category  Total Revenue
1               Mobiles      282295460
9               Gifting       83797316
0              Fashion        77176599
4      Sports / Fitness       66999950
8          Pet Supplies       62901594
6              Kitchen        51719318
3          Video Games        38954906
5  Medicines/Healthcare       32812708
7           Household         23792516
2                 Books       20206136


In [19]:
max_category = cat_revenue["Total Revenue"].sort_values(ascending=False).iloc[0]
cat_revenue[cat_revenue["Total Revenue"] == max_category]["Category"]

1    Mobiles
Name: Category, dtype: object

In [20]:
max_category = cat_revenue.sort_values(by="Total Revenue", ascending=False).iloc[0]["Category"]
print("Question 4 Asnwer: Top Revenue-Generating Category:", max_category)

Question 4 Asnwer: Top Revenue-Generating Category: Mobiles


## Question 5
What fraction of total sale value did Mumbai achieve? (across all categories and days) (FLOAT between 0 and 1)

In [21]:
#merged_skumaster_sales                      = pd.merge(skumaster, sales, how="inner", on="SKU")
merged_skumaster_sales                       = pd.merge(sales, skumaster, how="left", on="SKU")
merged_skumaster_sales["Total Sale Value"]   = merged_skumaster_sales["Price"] * merged_skumaster_sales["Sales"]
merged_skumaster_sales.head(5)

,Date,SKU,Product Name,City,Sales,Description,Category,Price,Total Sale Value
0,2025-01-01,F001,Prateek England,Mumbai,23,Prateek England,Fashion,1104,25392
1,2025-01-01,F002,Lucy Phillepe,Mumbai,75,Lucy Phillepe,Fashion,2485,186375
2,2025-01-01,F003,Badlands,Mumbai,33,Badlands,Fashion,1425,47025
3,2025-01-01,F004,Allen Dolly,Mumbai,3,Allen Dolly,Fashion,2640,7920
4,2025-01-01,F005,Bucchi,Mumbai,3,Bucchi,Fashion,11299,33897


In [22]:
city_name ="Mumbai"
total_sales  = merged_skumaster_sales  ["Total Sale Value"].sum()
city_sales = merged_skumaster_sales[merged_skumaster_sales["City"] == city_name]["Total Sale Value"].sum()
if total_sales != 0:
  fraction = city_sales / total_sales #print(fraction)
else:
    fraction = 0
print(f"Question 5 Asnwer: Fraction of total sales from {city_name}: {fraction:.4f}")

Question 5 Asnwer: Fraction of total sales from Mumbai: 0.3425


## Question 6
What is the no. of units of household category SKUs are in stock at the end of 15th Jan 2025 in Nasik DC?(INTEGER)

In [23]:
from openpyxl.utils import get_column_letter

In [24]:
household_skus = skumaster[skumaster["Category"].str.strip() == "Household"]["SKU"]
household_skus

35    H001
36    H002
37    H003
38    H004
39    H005
Name: SKU, dtype: object

In [25]:
#Opening Stock (as of Jan 1)
col = "Nashik"
os=openingstock.loc[openingstock["SKU"].isin(household_skus), col].sum()
print("Household Items Opening Stock (as of Jan 1) in Nasik DC :", os)

Household Items Opening Stock (as of Jan 1) in Nasik DC : 674


In [26]:
#Inward Stock Transfers (st) to Nasik (between Jan 1 and Jan 15)
household = stocktransfer[stocktransfer["City"].isin(household_skus)]
household.head()

,City,Pune,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93
36,H001,43,76,68,75,78,76,69,75,74,...,67,80,71,73,67,69,72,70,71,73
37,H002,26,76,33,20,46,63,24,44,44,...,86,98,97,81,80,84,76,84,80,81
38,H003,28,50,27,65,52,52,69,76,66,...,88,84,89,91,94,92,87,87,95,89
39,H004,37,35,42,78,46,40,63,72,33,...,81,78,91,91,95,92,89,84,82,84
40,H005,61,56,43,48,67,59,65,32,65,...,81,80,85,86,94,91,79,93,90,97


In [27]:
col = "Nasik"
if col in stocktransfer.columns:
    excel_col_index = stocktransfer.columns.get_loc(col) + 1
print(excel_col_index)
print(get_column_letter(excel_col_index))

64
BL


In [28]:
household.iloc[:, excel_col_index-1 : excel_col_index + 14]

,Nasik,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77
36,75,75,69,74,75,69,77,78,71,69,69,69,77,77,74
37,49,64,63,80,94,90,78,82,88,91,94,80,88,85,84
38,48,52,70,85,98,96,81,98,79,90,92,86,91,91,95
39,50,66,51,96,79,89,95,77,87,78,92,95,93,81,94
40,58,59,51,78,79,82,85,98,84,83,91,95,95,92,90


In [29]:
st = household.iloc[:, excel_col_index-1 : excel_col_index + 14].sum().sum()
print(" Stock Transfered for household items at Nasick DC (from Jan 1 to 15) :", st)

 Stock Transfered for household items at Nasick DC (from Jan 1 to 15) : 6003


In [30]:
#household items units_sold (us) between Jan 1 and Jan 15 )
from datetime import datetime
start_date = "2025-01-01"
end_date = "2025-01-15"
sales["Date"] = pd.to_datetime(sales["Date"])
us = sales[
    (sales["SKU"].isin(household_skus)) &
    (sales["City"] == "Nasik") &
    (sales["Date"] >= start_date) &
    (sales["Date"] <= end_date)
]["Sales"].sum()
print("Household items Units Sold at Nasik DC (from Jan 1 to 15) :", us )

Household items Units Sold at Nasik DC (from Jan 1 to 15) : 5387


In [31]:
# no. of units of household category SKUs are in stock at the end of 15th Jan 2025 in Nasik DC?(
final_units_in_stock = os + st - us
print("Question 6 Answer: Total units of Household category SKUs in stock at Nasik DC as of 15 Jan 2025:", int(final_units_in_stock))

Question 6 Answer: Total units of Household category SKUs in stock at Nasik DC as of 15 Jan 2025: 1290


## Question 7
Based on the sales and stock data of Jan 2025, how many average days of inventory of SKU M003 are available in Pune? (FLOAT)

In [32]:
#Opening Stock (as of Jan 1)
col = "Pune"
sku = "M003"
os_m003  = openingstock.loc[openingstock["SKU"]==sku, col].sum()
print("Opening Stock (as of Jan 1) in Pune DC :", os_m003)

Opening Stock (as of Jan 1) in Pune DC : 70


In [33]:
#Inward Stock Transfers (st) to Pune (between Jan 1 and Jan 31)
household = stocktransfer[stocktransfer["City"]==sku]
household.head()

,City,Pune,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93
8,M003,26,26,29,20,19,20,30,27,17,...,19,23,25,26,19,20,19,23,20,22


In [34]:
col = "Pune"
if col in stocktransfer.columns:
    excel_col_index = stocktransfer.columns.get_loc(col) + 1
print(excel_col_index)
print(get_column_letter(excel_col_index))

2
B


In [35]:
household.iloc[:, excel_col_index-1 : excel_col_index + 30]

,Pune,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
8,26,26,29,20,19,20,30,27,17,30,...,9,13,12,13,14,13,14,7,12,7


In [36]:
st_m003 = household.iloc[:, excel_col_index-1 : excel_col_index + 30].sum().sum()
print(" Stock Transfered for sku items at Pune DC (from Jan 1 to 31) :", st_m003)

 Stock Transfered for sku items at Pune DC (from Jan 1 to 31) : 511


In [37]:
#sku units_sold (us) between Jan 1 and Jan 30 )
from datetime import datetime
start_date = "2025-01-01"
end_date = "2025-01-31"
sales["Date"] = pd.to_datetime(sales["Date"])
us_m003 = sales[
    (sales["SKU"] == sku) &
    (sales["City"] == "Pune") &
    (sales["Date"] >= start_date) &
    (sales["Date"] <= end_date)
]["Sales"].sum()
print("SKU Units Sold at Pune DC (from Jan 1 to 31) :", us_m003 )

SKU Units Sold at Pune DC (from Jan 1 to 31) : 449


In [38]:
#closing stock (cs) of SKU at Pune at the end of Jan (on 31st)
cs_m003 = os_m003 + st_m003 - us_m003
average_inventory = (os_m003 + cs_m003) / 2
average_daily_sales = us_m003 / 31
days_of_inventory = average_inventory / average_daily_sales if average_daily_sales > 0 else float("inf")
days_of_inventory

np.float64(6.973273942093541)

In [39]:
print(f"Question 7 Answer: Days of Inventory for M003 in Pune (with transfers): {days_of_inventory:.4f}")

Question 7 Answer: Days of Inventory for M003 in Pune (with transfers): 6.9733


## Question 8
Which SKU has the highest average days of inventory in Aurangabad? (STRING)

In [40]:
def get_doi(stocktransfer, sku, city, days):
    os = openingstock.loc[openingstock["SKU"] == sku, city].values[0]
    #print(os)
    us = sales[(sales["SKU"] == sku) & (sales["City"] == city)]["Sales"].sum()
    #print(us)
    if city in stocktransfer.columns:
                      excel_col_index = stocktransfer.columns.get_loc(col) + 1
    df = stocktransfer[stocktransfer["City"]==sku]
    st = df.iloc[:, excel_col_index-1 : excel_col_index + days-1].sum().sum()  # for 31 days
    #print(st)
    cs = max(0, os + st - us)
    #print(cs)
    average_inventory = (os + cs) / 2
    #print(average_inventory)
    average_daily_sales = us / days
    #print(average_daily_sales)
    return average_inventory / average_daily_sales if average_daily_sales > 0 else float("inf")

In [41]:
city = "Aurangabad"
days = 31
sku="F001"
print(get_doi(stocktransfer, sku, city, days) )

15.301705756929637


In [42]:
skus = openingstock["SKU"].unique()
skus

array(['F001', 'F002', 'F003', 'F004', 'F005', 'M001', 'M002', 'M003',
       'M004', 'M005', 'B001', 'B002', 'B003', 'B004', 'B005', 'V001',
       'V002', 'V003', 'V004', 'V005', 'S001', 'S002', 'S003', 'S004',
       'S005', 'ME001', 'ME002', 'ME003', 'ME004', 'ME005', 'K001',
       'K002', 'K003', 'K004', 'K005', 'H001', 'H002', 'H003', 'H004',
       'H005', 'P001', 'P002', 'P003', 'P004', 'P005', 'G001', 'G002',
       'G003', 'G004', 'G005'], dtype=object)

In [43]:
stocktransfer.head()

,City,Pune,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93
0,SKU,2025-01-01 00:00:00,2025-01-02 00:00:00,2025-01-03 00:00:00,2025-01-04 00:00:00,2025-01-05 00:00:00,2025-01-06 00:00:00,2025-01-07 00:00:00,2025-01-08 00:00:00,2025-01-09 00:00:00,...,2025-01-22 00:00:00,2025-01-23 00:00:00,2025-01-24 00:00:00,2025-01-25 00:00:00,2025-01-26 00:00:00,2025-01-27 00:00:00,2025-01-28 00:00:00,2025-01-29 00:00:00,2025-01-30 00:00:00,2025-01-31 00:00:00
1,F001,20,27,23,19,20,19,19,19,24,...,55,49,47,59,52,58,52,52,54,55
2,F002,22,27,27,16,28,16,22,25,26,...,41,57,46,47,52,54,54,53,51,52
3,F003,26,28,30,20,18,27,27,25,28,...,58,53,45,46,57,44,44,52,41,51
4,F004,25,29,22,25,24,18,21,21,28,...,56,55,51,51,42,52,48,49,51,48


In [44]:
dois = {sku: get_doi(stocktransfer, sku, city, days) for sku in skus}
for sku, doi in sorted(dois.items(), key=lambda x: x[1], reverse=True):
    print(f"SKU: {sku} → DOI: {doi:.2f}")

SKU: V003 → DOI: 44.38
SKU: ME003 → DOI: 44.20
SKU: P001 → DOI: 33.26
SKU: S002 → DOI: 33.00
SKU: S005 → DOI: 32.50
SKU: P005 → DOI: 30.97
SKU: ME001 → DOI: 29.99
SKU: ME002 → DOI: 29.73
SKU: P002 → DOI: 29.69
SKU: V004 → DOI: 26.10
SKU: V002 → DOI: 24.72
SKU: P003 → DOI: 24.48
SKU: ME005 → DOI: 24.34
SKU: P004 → DOI: 23.15
SKU: F005 → DOI: 22.80
SKU: ME004 → DOI: 21.42
SKU: B005 → DOI: 21.39
SKU: K004 → DOI: 17.95
SKU: F003 → DOI: 17.80
SKU: M004 → DOI: 17.42
SKU: K001 → DOI: 16.22
SKU: B002 → DOI: 15.76
SKU: S004 → DOI: 15.62
SKU: F001 → DOI: 15.30
SKU: K005 → DOI: 15.23
SKU: F002 → DOI: 13.52
SKU: F004 → DOI: 13.39
SKU: M001 → DOI: 12.67
SKU: B001 → DOI: 12.28
SKU: B004 → DOI: 11.19
SKU: K002 → DOI: 10.19
SKU: V001 → DOI: 8.77
SKU: G001 → DOI: 8.68
SKU: V005 → DOI: 8.66
SKU: K003 → DOI: 8.63
SKU: S001 → DOI: 8.36
SKU: S003 → DOI: 8.26
SKU: M005 → DOI: 7.75
SKU: G003 → DOI: 7.10
SKU: B003 → DOI: 6.16
SKU: G005 → DOI: 4.84
SKU: M003 → DOI: 4.61
SKU: H001 → DOI: 4.55
SKU: M002 → DOI: 3

In [45]:
max_sku = max(dois, key=dois.get)
print(f"The SKU with the highest average DOI in {city} is: {max_sku}")

The SKU with the highest average DOI in Aurangabad is: V003


## Question 9
How many SKUs hold at least one weeks’ worth of inventory on average in Pune?(INTEGER)

In [46]:
def get_doi(stocktransfer, sku, city, days):
    os = openingstock.loc[openingstock["SKU"] == sku, city].values[0]
    #print(os)
    us = sales[(sales["SKU"] == sku) & (sales["City"] == city)]["Sales"].sum()
    #print(us)
    if city in stocktransfer.columns:
                      excel_col_index = stocktransfer.columns.get_loc(col) + 1
    df = stocktransfer[stocktransfer["City"]==sku]
    st = df.iloc[:, excel_col_index-1 : excel_col_index + days-1].sum().sum()  # for 31 days
    #print(st)
    cs = max(0, os + st - us)
    #print(cs)
    average_inventory = (os + cs) / 2
    #print(average_inventory)
    average_daily_sales = us / days
    #print(average_daily_sales)
    return average_inventory / average_daily_sales if average_daily_sales > 0 else float("inf")

In [47]:
city = "Pune"
days = 31
sku="F001"
print(get_doi(stocktransfer, sku, city, days) )

11.003486750348674


In [48]:
dois = {sku: get_doi(stocktransfer, sku, city, days) for sku in skus}
for sku, doi in sorted(dois.items(), key=lambda x: x[1], reverse=True):
    print(f"SKU: {sku} → DOI: {doi:.2f}")

SKU: ME003 → DOI: 27.37
SKU: ME004 → DOI: 22.50
SKU: F002 → DOI: 18.03
SKU: M005 → DOI: 17.02
SKU: B005 → DOI: 16.82
SKU: S005 → DOI: 16.70
SKU: M001 → DOI: 16.63
SKU: V003 → DOI: 16.20
SKU: ME001 → DOI: 15.60
SKU: H001 → DOI: 15.39
SKU: S002 → DOI: 14.96
SKU: M004 → DOI: 14.72
SKU: ME005 → DOI: 14.02
SKU: K005 → DOI: 12.34
SKU: K004 → DOI: 11.60
SKU: K003 → DOI: 11.40
SKU: P001 → DOI: 11.36
SKU: F001 → DOI: 11.00
SKU: ME002 → DOI: 10.58
SKU: P002 → DOI: 9.94
SKU: V002 → DOI: 9.29
SKU: V004 → DOI: 9.09
SKU: B001 → DOI: 9.07
SKU: S003 → DOI: 9.05
SKU: V001 → DOI: 7.74
SKU: B004 → DOI: 6.98
SKU: M003 → DOI: 6.97
SKU: B002 → DOI: 6.91
SKU: B003 → DOI: 6.85
SKU: H005 → DOI: 6.72
SKU: S004 → DOI: 6.68
SKU: S001 → DOI: 6.56
SKU: K001 → DOI: 6.30
SKU: H003 → DOI: 6.14
SKU: G001 → DOI: 6.05
SKU: K002 → DOI: 5.95
SKU: H002 → DOI: 5.65
SKU: M002 → DOI: 5.49
SKU: P003 → DOI: 5.25
SKU: F005 → DOI: 5.00
SKU: G002 → DOI: 4.94
SKU: V005 → DOI: 4.72
SKU: P005 → DOI: 4.48
SKU: H004 → DOI: 4.47
SKU: F00

In [49]:
count = sum(1 for doi in dois.values() if doi >= 7)
print("SKUs with ≥ 1 week's inventory:", count)

SKUs with ≥ 1 week's inventory: 25


## Question 10
What is the closing stock of K005 at the end of the month in Nasik? (INTEGER)

In [53]:
def get_doi_cs_os(stocktransfer, sku, city, days):
    os = openingstock.loc[openingstock["SKU"] == sku, "Nashik"].values[0]    # its not Nasik in openingstock, its Nashik. we need to figure out as to how to handle such a dirty dataset with improper metadata
    # print(os)
    us = sales[(sales["SKU"] == sku) & (sales["City"] == city)]["Sales"].sum()
    # print(us)
    if city in stocktransfer.columns:
                      excel_col_index = stocktransfer.columns.get_loc(col) + 1
    df = stocktransfer[stocktransfer["City"]==sku]
    st = df.iloc[:, excel_col_index-1 : excel_col_index + days-1].sum().sum()  # for 31 days
    print(st)
    cs = max(0, os + st - us)
    # print(cs)
    average_inventory = (os + cs) / 2
    #print(average_inventory)
    average_daily_sales = us / days if days else 0
    #print(average_daily_sales)
    doi=average_inventory / average_daily_sales if average_daily_sales > 0 else float("inf")
    return doi, cs, os

In [54]:
city = "Nasik"
days = 31
sku="K005"
print(get_doi_cs_os(stocktransfer, sku, city, days) )


838
(np.float64(7.459216965742251), np.int64(260), np.int64(35))


In [52]:
doi, cs,os = get_doi_cs_os(stocktransfer, sku, city, days)
print("Question 10 Answer: The closing stock of K005 at the end of the month in Nasik is : ", cs)

Question 10 Answer: The closing stock of K005 at the end of the month in Nasik is :  260
